In [ ]:
!pip install langchain unstructured python-magic-bin chromadb nltk
!pip install azure-storage-blob azure-identity

In [ ]:
import os, uuid
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.document_loaders import DirectoryLoader
import magic
import os
import nltk

os.environ['OPENAI_API_KEY'] = ''
connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

In [ ]:
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
# Create a unique name for the container
container_name = str(uuid.uuid4())

# Create the container
container_client = blob_service_client.create_container(container_name)
print("\nListing blobs...")

# List the blobs in the container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print("\t" + blob.name)

container_client = blob_service_client.get_container_client(container= container_name)
print("\nDownloading blob to \n\t" + os.getcwd() + "/private-data")

with open(file=os.getcwd()+'/private-data', mode="wb") as download_file:
 download_file.write(container_client.download_blob(blob.name).readall())
loader = DirectoryLoader('./private-data', glob='**/*.txt')

In [ ]:
documents = loader.load()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=0)

In [ ]:
texts = text_splitter.split_documents(documents)

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

In [ ]:
docsearch = Chroma.from_documents(texts, embeddings)

In [ ]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch)

In [ ]:
query = "How risky should I be as an investor?"
qa.run(query)

### Sources

In [ ]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
query = "Advise me on how to invest my money."
result = qa({"query": query})

In [ ]:
result['result']

In [ ]:
result['source_documents']